In [5]:
from glob import glob 
import os
import numpy as np
import struct
from open3d import *
from scipy.spatial.transform import Rotation as R

def convert_kitti_bin_to_pcd(binFilePath):
    size_float = 4
    list_pcd = []
    with open(binFilePath, "rb") as f:
        byte = f.read(size_float * 4)
        while byte:
            x, y, z, intensity = struct.unpack("ffff", byte)
            list_pcd.append([x, y, z])
            byte = f.read(size_float * 4)
    np_pcd = np.asarray(list_pcd)
    pcd = PointCloud()
    pcd.points = Vector3dVector(np_pcd)
    return pcd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
dataset_root = '/data/dataset/rellis-3d/Rellis-3D/'
os1_bins = 'os1_cloud_node_kitti_bin'

rellis_dirs = glob(os.path.join(dataset_root, "*"))

In [7]:
rellis_dirs

['/data/dataset/rellis-3d/Rellis-3D/00004',
 '/data/dataset/rellis-3d/Rellis-3D/00003',
 '/data/dataset/rellis-3d/Rellis-3D/00000',
 '/data/dataset/rellis-3d/Rellis-3D/00002',
 '/data/dataset/rellis-3d/Rellis-3D/00001']

In [11]:
pcd_list = []
pose_list = []

for run in rellis_dirs:
    # load pose.txt
    pose_txt = os.path.join(run, 'poses.txt')
    calib_txt = os.path.join(run, 'calib.txt')
    
    calib = {}
    
    for line in open(calib_txt, 'r').readlines():
        key, content = line.strip().split(":")
        values = [float(v) for v in content.strip().split()]

        pose = np.zeros((4, 4))
        pose[0, 0:4] = values[0:4]
        pose[1, 0:4] = values[4:8]
        pose[2, 0:4] = values[8:12]
        pose[3, 3] = 1.0

        calib[key] = pose
    
    Tr = calib['Tr']
    Tr_inv = np.linalg.inv(Tr)
    
    for line in open(pose_txt, 'r').readlines():
        values = [float(v) for v in line.strip().split()]

        pose = np.zeros((4, 4))
        pose[0, 0:4] = values[0:4]
        pose[1, 0:4] = values[4:8]
        pose[2, 0:4] = values[8:12]
        pose[3, 3] = 1.0
        
        transform_matrix = np.matmul(Tr_inv, np.matmul(pose, Tr))
        rotation_matrix = transform_matrix[:3, :3]
        quaternion = R.from_matrix(rotation_matrix).as_quat()
        translation = transform_matrix[:3, 3]
        
        q_dict = {'x': quaternion[0], 'y': quaternion[1], 'z': quaternion[2], 'w': quaternion[3]}
        t_dict = {'x': translation[0], 'y': translation[1], 'z': translation[2]}
        
        pose_list.append({'rotation': q_dict, 'position': t_dict})
        # pose_list.append({"rotation": quaternion, "position": translation})

        
    # pose_list.append(pose)

    # load bins and transform to pcd
    
    
    
    # break

In [9]:
quaternion

array([0.01407052, 0.00944946, 0.6018493 , 0.7984298 ])

In [17]:
translation

array([  52.3904 , -206.665  ,    1.09902])

In [16]:
R.from_matrix(rotation_matrix).as_euler('xyz', degrees=True)

array([ -2.3491508 ,  -2.73039231, -57.39364545])

In [12]:
pose_list

{'rotation': {'x': 0.014070520829566375,
  'y': 0.009449457855140769,
  'z': 0.6018493048380977,
  'w': 0.7984297980760281},
 'position': {'x': -0.136895, 'y': -0.401283, 'z': 0.166085}}